#### Libraray

In [80]:
import sys
import pandas as pd
import numpy as np
import os
import io
import matplotlib.pyplot as plt
import matplotlib.colors
### Key libraries ###
import plotly
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

import urllib
from collections import Counter
# Cufflinks connect Plotly with pandas to create graphs and charts of Dataframes directly
import cufflinks as cf   
cf.go_online()
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

#### Load Data

In [2]:
#df =pd.read_csv('session.csv')
#df.head(1)

C:\Users\JIIN\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning:

Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.



,Unnamed: 0,clnt_id,sess_id,sess_seq,sess_dt,tot_pag_view_ct,tot_sess_hr_v,dvc_ctg_nm,zon_nm,city_nm,...,clnt_age,hits_seq,pd_c,pd_add_nm,pd_bra_nm,pd_buy_am,pd_buy_ct,clac1_nm,clac2_nm,clac3_nm
0,0,5873599,8641867,9,20180509,82.0,1890.0,mobile,Daejeon,Daejeon,...,20,65,370753,색상:선택(04) 기본체크 4번 세트 / 1개,닥스(손수건),20000,1,패션잡화,패션액세서리,손수건


In [3]:
from pyspark.sql import SparkSession,SQLContext
spark = SparkSession.builder.getOrCreate()

In [4]:
path ='C:\\Users\\JIIN\\OneDrive\\Documents\\Github\\2021_Python\\Py_DS_ML_Bootcamp-master\\Refactored_Py_DS_ML_Bootcamp-master'
data_path = path + '\\session.csv'
df=spark.read.format('csv').option('delimiter',',').option('header' ,'true').option('inferschema','true').load(data_path)
df.show(1)

+---+-------+-------+--------+--------+---------------+-------------+----------+-------+-------+-----------+--------+--------+------+-----------------------------+------------+---------+---------+--------+------------+--------+
|_c0|clnt_id|sess_id|sess_seq| sess_dt|tot_pag_view_ct|tot_sess_hr_v|dvc_ctg_nm| zon_nm|city_nm|clnt_gender|clnt_age|hits_seq|  pd_c|                    pd_add_nm|   pd_bra_nm|pd_buy_am|pd_buy_ct|clac1_nm|    clac2_nm|clac3_nm|
+---+-------+-------+--------+--------+---------------+-------------+----------+-------+-------+-----------+--------+--------+------+-----------------------------+------------+---------+---------+--------+------------+--------+
|  0|5873599|8641867|       9|20180509|           82.0|         1890|    mobile|Daejeon|Daejeon|          F|      20|      65|370753|색상:선택(04) 기본체크 4번...|닥스(손수건)|    20000|        1|패션잡화|패션액세서리|  손수건|
+---+-------+-------+--------+--------+---------------+-------------+----------+-------+-------+-----------+-------

In [5]:
df.count()

3988524

In [6]:
# Spark SQL view 생성 
df.createOrReplaceTempView('ecom')

In [7]:
# data load check
spark.sql('select clnt_id  from ecom').show(3)

+-------+
|clnt_id|
+-------+
|5873599|
|5873599|
|5873599|
+-------+
only showing top 3 rows



#### Customer Retention Analysis with Spark SQL

In [8]:
df.columns

['_c0',
 'clnt_id',
 'sess_id',
 'sess_seq',
 'sess_dt',
 'tot_pag_view_ct',
 'tot_sess_hr_v',
 'dvc_ctg_nm',
 'zon_nm',
 'city_nm',
 'clnt_gender',
 'clnt_age',
 'hits_seq',
 'pd_c',
 'pd_add_nm',
 'pd_bra_nm',
 'pd_buy_am',
 'pd_buy_ct',
 'clac1_nm',
 'clac2_nm',
 'clac3_nm']

##### Identify the original pool of customers

https://sparkbyexamples.com/spark/spark-sql-date-and-time-functions/

In [19]:
spark.sql('SELECT COUNT(DISTINCT clnt_id)\
            FROM ecom' ).show()

+-----------------------+
|count(DISTINCT clnt_id)|
+-----------------------+
|                 671675|
+-----------------------+



In [39]:
df.printSchema() # date 변수 변환 필요 

root
 |-- _c0: integer (nullable = true)
 |-- clnt_id: integer (nullable = true)
 |-- sess_id: integer (nullable = true)
 |-- sess_seq: integer (nullable = true)
 |-- sess_dt: integer (nullable = true)
 |-- tot_pag_view_ct: double (nullable = true)
 |-- tot_sess_hr_v: string (nullable = true)
 |-- dvc_ctg_nm: string (nullable = true)
 |-- zon_nm: string (nullable = true)
 |-- city_nm: string (nullable = true)
 |-- clnt_gender: string (nullable = true)
 |-- clnt_age: integer (nullable = true)
 |-- hits_seq: integer (nullable = true)
 |-- pd_c: integer (nullable = true)
 |-- pd_add_nm: string (nullable = true)
 |-- pd_bra_nm: string (nullable = true)
 |-- pd_buy_am: integer (nullable = true)
 |-- pd_buy_ct: string (nullable = true)
 |-- clac1_nm: string (nullable = true)
 |-- clac2_nm: string (nullable = true)
 |-- clac3_nm: string (nullable = true)



In [61]:
spark.sql(" SELECT sess_dt FROM ecom ").show(1) 

+--------+
| sess_dt|
+--------+
|20180509|
+--------+
only showing top 1 row



    spark : from integer(numeric) to date 
    to_date(cast(col as string), 'yyyyMMdd')
    numeric > string > to_date method

     adding column : df.withColumn('nameofnewcol', valueofnewcol)

In [118]:
# sess_dt 문자&yyyyMMdd형태 > date 변수 추가 
df = spark.sql(" SELECT *, to_date(cast(sess_dt as string),'yyyyMMdd') date\
            FROM ecom ") 

In [120]:
df.show(1)

+---+-------+-------+--------+--------+---------------+-------------+----------+-------+-------+-----------+--------+--------+------+-----------------------------+------------+---------+---------+--------+------------+--------+----------+
|_c0|clnt_id|sess_id|sess_seq| sess_dt|tot_pag_view_ct|tot_sess_hr_v|dvc_ctg_nm| zon_nm|city_nm|clnt_gender|clnt_age|hits_seq|  pd_c|                    pd_add_nm|   pd_bra_nm|pd_buy_am|pd_buy_ct|clac1_nm|    clac2_nm|clac3_nm|      date|
+---+-------+-------+--------+--------+---------------+-------------+----------+-------+-------+-----------+--------+--------+------+-----------------------------+------------+---------+---------+--------+------------+--------+----------+
|  0|5873599|8641867|       9|20180509|           82.0|         1890|    mobile|Daejeon|Daejeon|          F|      20|      65|370753|색상:선택(04) 기본체크 4번...|닥스(손수건)|    20000|        1|패션잡화|패션액세서리|  손수건|2018-05-09|
+---+-------+-------+--------+--------+---------------+-------------+--

In [123]:
# 뷰 다시 생성 
df.createOrReplaceTempView('ecom')

In [125]:
# 데이터 시작 4월, 4월 고객 기준으로 retention 분석
spark.sql(" SELECT count(distinct clnt_id) users \
          from ecom \
         where month(date) = 4").show() 

+------+
| users|
+------+
|207292|
+------+



    4월 고객 207292명을 기준으로 9월까지 Retention 분석

In [134]:
# 데이터 시작 4월, 4월 고객 기준으로 retention 분석
april_pool = spark.sql(" SELECT distinct clnt_id \
          from ecom \
         where month(date) = 4") 
april_pool.show()

+-------+
|clnt_id|
+-------+
|1228390|
|2790126|
|5242833|
|1116547|
|6543472|
|4090737|
|3430560|
|6170791|
| 160009|
|5265133|
|3554625|
|5163615|
|1735744|
|3903475|
|6204450|
|4999861|
|4776942|
|2713813|
|5087172|
|1411904|
+-------+
only showing top 20 rows



In [135]:
april_pool.createOrReplaceTempView('april_pool')

In [136]:
# 4월 고객
retention_month = spark.sql ("select month(date) month,\
              count(distinct e.clnt_id) as number \
           from ecom e where e.clnt_id in (select clnt_id from april_pool)\
           group by 1 order by 1")
retention_month.show()

+-----+------+
|month|number|
+-----+------+
|    4|207292|
|    5| 90484|
|    6| 77534|
|    7| 69795|
|    8| 57288|
|    9| 16587|
+-----+------+



In [145]:
retention_month.shoW

DataFrame[month: int, number: bigint]

* Spark df to Pandas df 
    - retention_month.toPandas()

###### Retention graph by months : X months Y users(number)


In [138]:
import plotly.express as px

In [205]:
# spart dataframe to Pandas dataframe
data = retention_month.toPandas()
fig = px.line(data, x="month", y="number", title='Retention of customers who visited in April',
             width=600, height=400,
             labels = dict(month='Month', number='# of customers'))
fig.show()

In [149]:
print(fig)

Figure({
    'data': [{'hovertemplate': 'month=%{x}<br>number=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([4, 5, 6, 7, 8, 9]),
              'xaxis': 'x',
              'y': array([207292,  90484,  77534,  69795,  57288,  16587], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Retention'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'month'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'number'}}}
})


##### EVOLUTION OF CUSTOMER RETENTION OVER TIME
    분석 데이터시작기준 : 2018-04-01 기준
    one month interval customer retention trend : NEXT VISIT > LEAD
    1) visit log 
    2) time lapse
    3) time gaps

###### VISIT LOG


    c4.withColumn("dateDifference", datediff(col("end"),col("start")))
    .withColumn("monthDifference", months_between(col("end"),col("start")))
    .withColumn("yearDifference", year(col("end"))-year(col("start")))
    .show()

In [172]:
# visit log
visitlog = spark.sql('select distinct clnt_id,month(date) visit_month \
          from ecom \
          group by 1,2 \
          order by 1,2 ')
visitlog.show()

+-------+-----------+
|clnt_id|visit_month|
+-------+-----------+
|      8|          6|
|     14|          9|
|     32|          9|
|     56|          6|
|     56|          8|
|     58|          7|
|     59|          9|
|     63|          4|
|     63|          6|
|     63|          9|
|     64|          4|
|     68|          7|
|     68|          8|
|     71|          4|
|     71|          7|
|     73|          5|
|     73|          6|
|     78|          9|
|     89|          9|
|     93|          5|
+-------+-----------+
only showing top 20 rows



In [173]:
visitlog.createOrReplaceTempView('visitlog')

###### TIME LAPSE > TIME DIFF > SEGEMENT

In [174]:
# time lapse
time_lapse = spark.sql("select clnt_id, visit_month, \
lead(visit_month,1) over (partition by clnt_id order by clnt_id, visit_month) next_visit \
from visitlog")
time_lapse.show()

+-------+-----------+----------+
|clnt_id|visit_month|next_visit|
+-------+-----------+----------+
|    148|          9|      null|
|    463|          9|      null|
|   2366|          9|      null|
|   2659|          4|      null|
|   7240|          4|         5|
|   7240|          5|      null|
|   9900|          9|      null|
|  10206|          4|         6|
|  10206|          6|         7|
|  10206|          7|      null|
|  11748|          7|      null|
|  13840|          9|      null|
|  17679|          5|      null|
|  20135|          9|      null|
|  20924|          5|         7|
|  20924|          7|         8|
|  20924|          8|      null|
|  22373|          9|      null|
|  24354|          9|      null|
|  25591|          9|      null|
+-------+-----------+----------+
only showing top 20 rows



In [176]:
# time_diff
spark.sql("with time_lapse as ( \
            select clnt_id, visit_month,lead(visit_month,1) over (partition by clnt_id order by clnt_id, visit_month) next_visit from visitlog\
        ) select *, next_visit-visit_month as time_diff from time_lapse").show()

+-------+-----------+----------+---------+
|clnt_id|visit_month|next_visit|time_diff|
+-------+-----------+----------+---------+
|    148|          9|      null|     null|
|    463|          9|      null|     null|
|   2366|          9|      null|     null|
|   2659|          4|      null|     null|
|   7240|          4|         5|        1|
|   7240|          5|      null|     null|
|   9900|          9|      null|     null|
|  10206|          4|         6|        2|
|  10206|          6|         7|        1|
|  10206|          7|      null|     null|
|  11748|          7|      null|     null|
|  13840|          9|      null|     null|
|  17679|          5|      null|     null|
|  20135|          9|      null|     null|
|  20924|          5|         7|        2|
|  20924|          7|         8|        1|
|  20924|          8|      null|     null|
|  22373|          9|      null|     null|
|  24354|          9|      null|     null|
|  25591|          9|      null|     null|
+-------+--

In [178]:
# segment by time_diff (방문한 월간 차이)
segment = spark.sql("with time_lapse as ( \
            select clnt_id, visit_month,lead(visit_month,1) over (partition by clnt_id order by clnt_id, visit_month) next_visit from visitlog\
        ), time_diff as( \
          select *, next_visit-visit_month as time_diff from time_lapse \
          ) select clnt_id, visit_month, \
          case when time_diff=1 then 'retained'\
              when time_diff >1 then 'lagger' \
              when time_diff is null then 'lost' end  as segment \
          from time_diff ")
segment.show()

+-------+-----------+--------+
|clnt_id|visit_month| segment|
+-------+-----------+--------+
|    148|          9|    lost|
|    463|          9|    lost|
|   2366|          9|    lost|
|   2659|          4|    lost|
|   7240|          4|retained|
|   7240|          5|    lost|
|   9900|          9|    lost|
|  10206|          4|  lagger|
|  10206|          6|retained|
|  10206|          7|    lost|
|  11748|          7|    lost|
|  13840|          9|    lost|
|  17679|          5|    lost|
|  20135|          9|    lost|
|  20924|          5|  lagger|
|  20924|          7|retained|
|  20924|          8|    lost|
|  22373|          9|    lost|
|  24354|          9|    lost|
|  25591|          9|    lost|
+-------+-----------+--------+
only showing top 20 rows



In [180]:
segment.createOrReplaceTempView('segment')

In [190]:
spark.sql('select * from segment order by clnt_id').show(10)

+-------+-----------+-------+
|clnt_id|visit_month|segment|
+-------+-----------+-------+
|      8|          6|   lost|
|     14|          9|   lost|
|     32|          9|   lost|
|     56|          8|   lost|
|     56|          6| lagger|
|     58|          7|   lost|
|     59|          9|   lost|
|     63|          6| lagger|
|     63|          4| lagger|
|     63|          9|   lost|
+-------+-----------+-------+
only showing top 10 rows



In [193]:
spark.sql('select count(clnt_id) from segment where visit_month=9 and segment="retained"').show()

+--------------+
|count(clnt_id)|
+--------------+
|             0|
+--------------+



In [212]:
# monthly retention (the proportion of customers who returned)
mom_retention = spark.sql("select visit_month, \
          round(sum(case when segment='retained' then 1 else 0 end)/count(clnt_id),2) as retention\
          from segment\
          group by 1 \
          order by 1")
mom_retention.show()

+-----------+---------+
|visit_month|retention|
+-----------+---------+
|          4|     0.44|
|          5|     0.42|
|          6|     0.41|
|          7|     0.37|
|          8|     0.15|
|          9|      0.0|
+-----------+---------+



###### Retention graph by months : X months Y retention


In [217]:
# spark dataframe to Pandas dataframe
data = mom_retention.toPandas()
fig = px.line(data, x="visit_month", y="retention", text="retention", title='Month-to-Month Retention',
             width=700, height=500,
              labels = dict(visit_month='Month', retention='Retention'))
fig.update_traces(textposition='top center')
fig.show()

##### EVOLUTION OF CUSTOMER RETENTION OVER TIME
    분석 데이터시작기준 : 2018-04-01 기준
    one month interval customer retention trend : PREVIOUS VISIT > LAG
    1) visit log 
    2) time lapse
    3) time gaps

In [220]:
# time lapse
time_lapse2 = spark.sql("select clnt_id, visit_month, \
lag(visit_month,1) over (partition by clnt_id order by clnt_id, visit_month) pr_visit \
from visitlog")
time_lapse2.show()

+-------+-----------+----------+
|clnt_id|visit_month|next_visit|
+-------+-----------+----------+
|    148|          9|      null|
|    463|          9|      null|
|   2366|          9|      null|
|   2659|          4|      null|
|   7240|          4|      null|
|   7240|          5|         4|
|   9900|          9|      null|
|  10206|          4|      null|
|  10206|          6|         4|
|  10206|          7|         6|
|  11748|          7|      null|
|  13840|          9|      null|
|  17679|          5|      null|
|  20135|          9|      null|
|  20924|          5|      null|
|  20924|          7|         5|
|  20924|          8|         7|
|  22373|          9|      null|
|  24354|          9|      null|
|  25591|          9|      null|
+-------+-----------+----------+
only showing top 20 rows



In [223]:
spark.sql ('with time_lapse2 as (select clnt_id, visit_month, \
lag(visit_month,1) over (partition by clnt_id order by clnt_id, visit_month) pr_visit \
from visitlog) \
           select clnt_id, visit_month, pr_visit, visit_month-pr_visit as time_diff \
           from time_lapse2 \
           order by 1,2').show()

+-------+-----------+--------+---------+
|clnt_id|visit_month|pr_visit|time_diff|
+-------+-----------+--------+---------+
|      8|          6|    null|     null|
|     14|          9|    null|     null|
|     32|          9|    null|     null|
|     56|          6|    null|     null|
|     56|          8|       6|        2|
|     58|          7|    null|     null|
|     59|          9|    null|     null|
|     63|          4|    null|     null|
|     63|          6|       4|        2|
|     63|          9|       6|        3|
|     64|          4|    null|     null|
|     68|          7|    null|     null|
|     68|          8|       7|        1|
|     71|          4|    null|     null|
|     71|          7|       4|        3|
|     73|          5|    null|     null|
|     73|          6|       5|        1|
|     78|          9|    null|     null|
|     89|          9|    null|     null|
|     93|          5|    null|     null|
+-------+-----------+--------+---------+
only showing top

In [253]:
segment2=  spark.sql ('with time_lapse2 as ( \
            select clnt_id, visit_month, \
            lag(visit_month,1) over (partition by clnt_id order by clnt_id, visit_month) pr_visit \
            from visitlog \
          ), time_diff2 as (\
           select clnt_id, visit_month, pr_visit, visit_month-pr_visit as time_diff \
           from time_lapse2  \
          ) select clnt_id, visit_month,  \
           case when time_diff = 1 then "retained" \
                when  time_diff > 1 then "returning" \
               when time_diff is null then "new" end as segment2 \
           from time_diff2 ')
segment2.show()

+-------+-----------+---------+
|clnt_id|visit_month| segment2|
+-------+-----------+---------+
|    148|          9|      new|
|    463|          9|      new|
|   2366|          9|      new|
|   2659|          4|      new|
|   7240|          4|      new|
|   7240|          5| retained|
|   9900|          9|      new|
|  10206|          4|      new|
|  10206|          6|returning|
|  10206|          7| retained|
|  11748|          7|      new|
|  13840|          9|      new|
|  17679|          5|      new|
|  20135|          9|      new|
|  20924|          5|      new|
|  20924|          7|returning|
|  20924|          8| retained|
|  22373|          9|      new|
|  24354|          9|      new|
|  25591|          9|      new|
+-------+-----------+---------+
only showing top 20 rows



In [262]:
segment2_cnt = spark.sql('select visit_month, segment2 as segment, count(clnt_id) number \
                         from segment2 group by 1,2 order by  1,2')
segment2_cnt.show()

+-----------+---------+------+
|visit_month|  segment|number|
+-----------+---------+------+
|          4|      new|207292|
|          5|      new|128962|
|          5| retained| 90484|
|          6|      new| 97494|
|          6| retained| 92006|
|          6|returning| 27663|
|          7|      new| 76289|
|          7| retained| 89961|
|          7|returning| 42598|
|          8|      new| 62600|
|          8| retained| 77071|
|          8|returning| 46425|
|          9|      new| 99038|
|          9| retained| 28820|
|          9|returning| 29415|
+-----------+---------+------+



In [263]:
segment2_cnt.createOrReplaceTempView('segment2_cnt')

In [264]:
segment2_cnt_filtered=spark.sql('select * from segment2_cnt where visit_month!=4')
segment2_cnt_filtered.show()

+-----------+---------+------+
|visit_month|  segment|number|
+-----------+---------+------+
|          5|      new|128962|
|          5| retained| 90484|
|          6|      new| 97494|
|          6| retained| 92006|
|          6|returning| 27663|
|          7|      new| 76289|
|          7| retained| 89961|
|          7|returning| 42598|
|          8|      new| 62600|
|          8| retained| 77071|
|          8|returning| 46425|
|          9|      new| 99038|
|          9| retained| 28820|
|          9|returning| 29415|
+-----------+---------+------+



In [265]:
# spark dataframe to Pandas dataframe
# time series data ORDER BY 체크
data = segment2_cnt_filtered.toPandas()
#데이터 시작이 4월 , new customer 잘못된 분류 제거
fig = px.line(data, x="visit_month", y="number", title='Customer Type by month',
             width=700, height=500,
              labels = dict(visit_month='Month', number='number'),
             color = 'segment')
fig.update_traces(textposition='top center')
fig.show()

##### COHORT TABLES
    첫 유입일자 기준으로 retention 패턴 파악
    defining each user by their first visit > seeing how they return over time
    1) classify each customer according to their initial visit
    2) length of the time that they are retained

In [279]:
# min(visit_month) 유입월
first_visit = spark.sql('select clnt_id, min(visit_month) as first_visit from visitlog group by 1')
first_visit.show()

+-------+-----------+
|clnt_id|first_visit|
+-------+-----------+
|2955100|          5|
|3316844|          4|
| 181209|          4|
| 368691|          4|
|6474458|          4|
|4089626|          6|
| 757731|          4|
|5523878|          9|
|3475112|          4|
|3311574|          5|
| 641602|          4|
|4999861|          4|
|3319786|          4|
|3060453|          5|
|2330670|          4|
|  31367|          6|
|3776028|          4|
|6131634|          5|
|2468088|          6|
|1211670|          6|
+-------+-----------+
only showing top 20 rows



In [272]:
# calculate the number of new users
newuser=spark.sql('with first as ( \
          select clnt_id, min(visit_month) as first_visit from visitlog group by 1 \
         ) select first_visit, count(distinct clnt_id) as new from first group by 1 order by 1')
newuser.show()

+-----------+------+
|first_visit|   new|
+-----------+------+
|          4|207292|
|          5|128962|
|          6| 97494|
|          7| 76289|
|          8| 62600|
|          9| 99038|
+-----------+------+



In [278]:
# SELF JOIN to track the visit pattern  of each customer
visit_tracker =spark.sql('select a.clnt_id, a.visit_month from visitlog a\
            left join visitlog visit_tracker  \
          on a.clnt_id = visit_tracker.clnt_id \
              and a.visit_month < visit_tracker.visit_month \
              order by 1 ')
visit_tracker.show()

+-------+-----------+
|clnt_id|visit_month|
+-------+-----------+
|      8|          6|
|     14|          9|
|     32|          9|
|     56|          6|
|     56|          8|
|     58|          7|
|     59|          9|
|     63|          9|
|     63|          4|
|     63|          4|
|     63|          6|
|     64|          4|
|     68|          8|
|     68|          7|
|     71|          7|
|     71|          4|
|     73|          5|
|     73|          6|
|     78|          9|
|     89|          9|
+-------+-----------+
only showing top 20 rows



In [280]:
first_visit.createOrReplaceTempView('first_visit')
visit_tracker.createOrReplaceTempView('visit_tracker')
newuser.createOrReplaceTempView('newuser')

In [ ]:
spark.sql (' select f.first_month, new, 
           (t.visit_month-l.visit_month) as retention_month,
           count(distinct visit_tracker)
           from first_visit f
           left join visitlog visit_tracker on f.clnt_id = l.clnt_id and vis
           left join newuser n on n.first_month = f.first_month
           group by 1,2
           ')

In [ ]:
# to track the visit pattern of each customer